<img src="../imagenes/flow.png" width="350" height="150"> <img src="../imagenes/cp.png" width="310" height="150"><br>
# **Datathon2021**
### Desafío: **Recomendador**
### Participante: **Miriam Lanabere**
---

<a id="indice"></a> 
### Indice

<a href='#Objetivo'>1. Objetivo</a>

<a href='#Workflow'>2. Workflow</a>

<a href='#Dataset'>3. Dataset</a>

<a href='#Data_wrangling'>4. Data wrangling</a>

<a href='#EDA'>5. EDA</a>

<a href='#Modeling'>6. Modeling</a>

<a href='#Conclusión'>7. Conclusión</a>

<a id="Objetivo"></a> 
### 1. Objetivo

La personalización de las plataformas digitales de contenidos audiovisuales es un aspecto central en la calidad de la experiencia de los usuarios. Dada la amplia oferta de posibilidades, los consumidores tienden a valorar las recomendaciones sobre qué contenidos ver que sean acordes a sus preferencias.

En este sentido,se plantea el siguiente desafío: **desarrollar un sistema de recomendación para predecir qué nuevos contenidos, no previamente vistos, son más probables a ser elegidos para ver en Flow por un grupo de usuarios en base a su historial de visualizaciones.** 

Partiendo de las visualizaciones de un conjunto de usuarios de Flow correspondientes a un trimestre del 2021, el desafío consiste en predecir qué nuevas visualizaciones tuvo cada perfil durante el mes siguiente.

Algunas consideraciones:

<ul>
    <li> Sistema de recomendación híbrido: combinar los beneficios de los modelos de recomendación basados en contenido y de filtro colaborativo. </li>
    <li> Excluir de la recomendación los contenidos ya vistos por el usuario y sus derivados (sagas, episodios, etc.) </li>
</ul>

<a id="Workflow"></a> 
### 2. Workflow

Para el desafío se ofrecen 3 bases de datos:

<ul>
    <li> train.csv: contiene los registros de visualizaciones de contenidos de Flow correspondiente a una muestra aleatoria de más de 113 mil perfiles.</li>
    <li> metadata.csv: contiene la metadata asociada a cada uno de los contenidos. </li>
    <li> iso_3166_1.json: contiene un diccionario de los códigos de nombres de países propios del estándar internacional de normalización ISO 3166.</li>
</ul>

El resultado a entregar deber ser una lista de recomendación con los primeros 20 resultados:

**account_id: [content_id X 20]**<br>
**account_id: [content_id X 20]**<br>
**account_id: [content_id X 20]**<br>
...<br>
**(X 113mil account_id)**

Se propone el siguiente esquema de trabajo para armar el modelo de recomendación:




<a id="Dataset"></a> 
### 3. Dataset

Importamos los archivos train y metadata, los unimos por la variable "asset_id" y armamos un único dataframe con la siguiente información:

**train.csv**<br>
<ul>
    <li>customer_id: código de identificación de cada cliente de Flow (puede tener asociados uno o más account_id)</li>
    <li>account_id: código de identificación de cada perfil de Flow (se corresponde con un único customer_id)</li>
    <li>device_type: indica el tipo de dispositivo desde el que se efectuó la visualización. Las categorías posibles son:</li>
    <ul>
        <li>CLOUD: cliente web</li>
        <li>PHONE: teléfono celular</li>
        <li>STATIONARY: smart TV</li>
        <li>STB: set-top box, el decodificador Flow</li>
        <li>TABLET</li>
    </ul>
    <li>asset_id: código de identificación de cada activo (video) disponible en la plataforma</li>
    <li>tunein: fecha y hora de inicio de cada visualización</li>
    <li>tuneout: fecha y hora de finalización de cada visualización</li>
    <li>resume: variable dummy que indica si se reanuda un consumo anterior del mismo asset_id</li>
</ul>
    
**metadata.csv**<br>
<ul>
    <li>asset_id: código de identificación de cada activo (video) disponible en Flow</li>
    <li>content_id: código de identificación que agrupa los distintos asset_id asociados a un mismo contenido (por ejemplo, cada episodio de una misma serie tiene su propio asset_id, mientras que la serie se identifica con un content_id único)</li>
    <li>title: título</li>
    <li>reduced_title: título reducido</li>
    <li>episode_title: título del episodio (válido para contenidos episódicos, como las series)</li>
    <li>show_type: tipo de show - las categorías son autorreferenciales con excepción de “Rolling”, que indica que se trata de una serie incompleta, y “Web”, la cual remite a contenidos pensados íntegramente en formato digital (series web) -</li>
    <li>released_year: año de lanzamiento</li>
    <li>country_of_origin: país de origen – expresado con el código de dos letras propio del estándar internacional de normalización ISO 3166 -</li>
    <li>category: categoría o género al que pertenece el contenido - puede haber una o más -</li>
    <li>keywords: palabras clave o tags asociadas al contenido - puede haber una o más -</li>
    <li>description: descripción (sinopsis)</li>
    <li>reduced_desc: descripción (sinopsis) reducida</li>
    <li>cast_first_name: nombre y apellido de los actores y actrices principales</li>
    <li>credits_first_name: nombre y apellido del director o directora</li>
    <li>run_time_min: duración total, expresada en minutos</li>
    <li>audience: audiencia target</li>
    <li>made_for_tv: variable dummy que indica si el contenido fue hecho para TV</li>
    <li>close_caption: variable dummy que indica si el contenido posee subtítulos</li>
    <li>sex_rating: variable dummy que indica si el contenido tiene escenas de sexo explícitas</li>
    <li>violence_rating: variable dummy que indica si el contenido tiene escenas de violencia explícitas</li>
    <li>language_rating: variable dummy que indica si el contenido posee lenguaje que puede ser considerado ofensivo o inapropiado</li>
    <li>dialog_rating: variable dummy que indica si el contenido posee diálogos que pueden ser considerado ofensivos o inapropiados</li>
    <li>fv_rating: variable dummy que indica si el contenido tiene rating de FV, que corresponde a público infantil con violencia ficticia</li>
    <li>pay_per_view: variable dummy que indica si se trata de un alquiler</li>
    <li>pack_premium_1: variable dummy que indica si se trata de un contenido exclusivo del pack premium 1</li>
    <li>pack_premium_2: variable dummy que indica si se trata de un contenido exclusivo del pack premium 2</li>
    <li>create_date: fecha de creación del activo</li>
    <li>modify_date: fecha de modificación del activo</li>
    <li>start_vod_date: fecha desde la cual el activo se encuentra disponible en la plataforma</li>
    <li>end_vod_date: fecha de finalización de la disponibilidad del activo en la plataforma</li>
</ul>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("data/train.csv")
data.head(5)

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume
0,0,90627,STATIONARY,18332.0,2021-02-18 22:52:00.0,2021-02-18 23:35:00.0,0
1,0,90627,STATIONARY,24727.0,2021-03-24 23:17:00.0,2021-03-25 00:01:00.0,0
2,1,3387,STB,895.0,2021-03-15 10:05:00.0,2021-03-15 10:23:00.0,0
3,1,3387,STB,895.0,2021-03-15 10:23:00.0,2021-03-15 11:18:00.0,1
4,1,3387,STB,26062.0,2021-03-16 09:24:00.0,2021-03-16 09:44:00.0,0


In [3]:
meta_data = pd.read_csv("data/metadata.csv", sep=";")
meta_data.head()

,asset_id,content_id,title,reduced_title,episode_title,show_type,released_year,country_of_origin,category,keywords,...,language_rating,dialog_rating,fv_rating,pay_per_view,pack_premium_1,pack_premium_2,create_date,modify_date,start_vod_date,end_vod_date
0,15188,0.0,Ep:17 Tiempos Compulsivos,Tiempos_Compul_E17,Episodio 17,Serie,2012.0,AR,Drama,"Trastornos,Médicos,Tragicómica,Telenovela,Enfe...",...,N,N,N,N,N,N,2017-12-01T10:18:15.0Z,2019-01-26T06:37:18.0Z,2017-12-01T00:00:00.0Z,2020-12-01T23:59:59.0Z
1,24940,1.0,7 Cajas,7_Cajas,NaN,Película,2012.0,PY,Suspenso/Acción,"Latinoamérica,Pobreza,Crimen,Pandillas",...,N,N,N,Y,N,N,2017-12-19T20:58:15.0Z,2019-09-17T19:02:03.0Z,2017-12-15T00:00:00.0Z,2022-12-14T23:59:59.0Z
2,21939,2.0,La Maldición de las Hormigas Gigantes,La_Maldicion_de_las,NaN,Película,2016.0,FI,Terror/Comedia,"Criaturas,Plagas,Adolescentes,Fantasía,Video J...",...,N,N,N,N,N,N,2018-02-16T13:51:07.0Z,2020-04-28T14:16:38.0Z,2018-01-25T00:00:00.0Z,2020-12-01T23:59:59.0Z
3,9005,3.0,Una Mujer Fantástica,Una_Mujer_Fantastic,NaN,Película,2017.0,CL,Drama,"LGBT,Mujeres,Latinoamérica",...,N,N,N,N,Y,N,2018-05-26T11:58:44.0Z,2019-11-15T03:00:23.0Z,2018-05-27T00:00:00.0Z,2021-04-30T23:59:59.0Z
4,7391,4.0,Star Trek,Star_Trek,NaN,Película,2009.0,US,Ciencia Ficción/Aventura,"Fantasía,Galaxia,Futurismo,Aliens,Criaturas",...,N,N,N,Y,N,N,2019-05-03T20:07:24.0Z,2020-04-09T04:37:29.0Z,2019-05-02T00:00:00.0Z,2020-12-31T23:59:59.0Z


In [4]:
df = data.join(meta_data.set_index('asset_id'), on='asset_id')
pd.options.display.max_columns = 0
df.head()

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume,content_id,title,reduced_title,episode_title,show_type,released_year,country_of_origin,category,keywords,description,reduced_desc,cast_first_name,credits_first_name,run_time_min,audience,made_for_tv,close_caption,sex_rating,violence_rating,language_rating,dialog_rating,fv_rating,pay_per_view,pack_premium_1,pack_premium_2,create_date,modify_date,start_vod_date,end_vod_date
0,0,90627,STATIONARY,18332.0,2021-02-18 22:52:00.0,2021-02-18 23:35:00.0,0,2040.0,T:5 Ep:08 This is Us,This_is_Us_T5_E08,En la habitación,Serie,2020.0,US,Drama/Comedia,"Sociedad,Tragicómica,Adicciones,Familia,Amigos...",Los Pearson navegan juntos por grandes hitos f...,Los Pearson navegan juntos por grandes hitos f...,"Milo Ventimiglia, Mandy Moore, , Chrissy Metz",Ken Olin,43.0,Mujeres,N,N,N,N,N,N,N,N,Y,N,2021-02-18T19:05:36.0Z,2021-03-25T23:13:43.0Z,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z
1,0,90627,STATIONARY,24727.0,2021-03-24 23:17:00.0,2021-03-25 00:01:00.0,0,2040.0,T:5 Ep:10 This is Us,This_is_Us_T5_E10,De esto me encargo yo,Serie,2020.0,US,Drama/Comedia,"Sociedad,Tragicómica,Adicciones,Familia,Amigos...",Beth se desenvuelve entre los escrúpulos que c...,Beth se desenvuelve entre los escrúpulos que c...,"Milo Ventimiglia, Mandy Moore, , Chrissy Metz",Ken Olin,42.0,Mujeres,N,N,N,N,N,N,N,N,Y,N,2021-03-17T20:31:21.0Z,2021-03-25T23:15:49.0Z,2021-03-18T00:00:00.0Z,2021-06-30T23:59:59.0Z
2,1,3387,STB,895.0,2021-03-15 10:05:00.0,2021-03-15 10:23:00.0,0,1983.0,T:1 Ep:02 Big Little Lies,Big_Little_L_T1_E02,Serious Mothering,Serie,2017.0,US,Drama/Crimen,"Crimen,Abusos,Mujeres,De Libros,Feminismo,Gold...",Disfrutá esta temporada sin cargo hasta el 28/...,Jane evita las preguntas de Ziggy. Madeline se...,"Reese Witherspoon, Nicole Kidman, Shailene Woo...",Jean-Marc Vallée,54.0,Mujeres,N,N,N,N,N,N,N,N,N,N,2021-03-05T18:37:58.0Z,2021-03-08T02:06:03.0Z,2021-03-08T00:00:00.0Z,2021-03-28T23:59:00.0Z
3,1,3387,STB,895.0,2021-03-15 10:23:00.0,2021-03-15 11:18:00.0,1,1983.0,T:1 Ep:02 Big Little Lies,Big_Little_L_T1_E02,Serious Mothering,Serie,2017.0,US,Drama/Crimen,"Crimen,Abusos,Mujeres,De Libros,Feminismo,Gold...",Disfrutá esta temporada sin cargo hasta el 28/...,Jane evita las preguntas de Ziggy. Madeline se...,"Reese Witherspoon, Nicole Kidman, Shailene Woo...",Jean-Marc Vallée,54.0,Mujeres,N,N,N,N,N,N,N,N,N,N,2021-03-05T18:37:58.0Z,2021-03-08T02:06:03.0Z,2021-03-08T00:00:00.0Z,2021-03-28T23:59:00.0Z
4,1,3387,STB,26062.0,2021-03-16 09:24:00.0,2021-03-16 09:44:00.0,0,729.0,T:1 Ep:02 Dime quién soy,Dime_quien_s_T1_E02,Capítulo 2: Pierre,Serie,2020.0,ES,Drama,"30s,Espionaje,Historia,Europa",Disfrutá esta temporada sin cargo hasta el 28/...,"Buenos Aires, 1936. Con la ayuda de la estrell...","Irene Escolar, María Pía Calzone, Pablo Derqui...",Eduard Cortés,49.0,General,N,N,N,N,N,N,N,N,N,N,2021-03-06T13:47:10.0Z,2021-03-08T02:06:06.0Z,2021-03-08T00:00:00.0Z,2021-03-28T23:59:00.0Z


In [5]:
df.shape

(3657801, 36)

**¡Nuestro dataframe es muy grande!**

<a id="Data_wrangling"></a> 
### 4. Data wrangling


En este paso buscamos obtener un dataframe limpio y consistente para poder correr el modelo.<br>
Para ello se realizaran los siguientes pasos:<br>
<ul>
    <li>Data cleaning</li>
    <li>EDA</li>
    <li>Feature engineering</li>
    <li>Feature selection</li>
</ul>

**Data cleaning**

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3657801 entries, 0 to 3657800
Data columns (total 36 columns):
 #   Column              Dtype  
---  ------              -----  
 0   customer_id         int64  
 1   account_id          int64  
 2   device_type         object 
 3   asset_id            float64
 4   tunein              object 
 5   tuneout             object 
 6   resume              int64  
 7   content_id          float64
 8   title               object 
 9   reduced_title       object 
 10  episode_title       object 
 11  show_type           object 
 12  released_year       float64
 13  country_of_origin   object 
 14  category            object 
 15  keywords            object 
 16  description         object 
 17  reduced_desc        object 
 18  cast_first_name     object 
 19  credits_first_name  object 
 20  run_time_min        float64
 21  audience            object 
 22  made_for_tv         object 
 23  close_caption       object 
 24  sex_rating          obje

In [7]:
df.isnull().sum()

customer_id                 0
account_id                  0
device_type                29
asset_id                   22
tunein                      0
tuneout                     0
resume                      0
content_id                142
title                      22
reduced_title              22
episode_title          885156
show_type                  35
released_year              22
country_of_origin          40
category                   22
keywords                  442
description                42
reduced_desc               22
cast_first_name       1016037
credits_first_name     842844
run_time_min               22
audience                  131
made_for_tv                22
close_caption              22
sex_rating                 22
violence_rating            22
language_rating            22
dialog_rating              22
fv_rating                  22
pay_per_view               22
pack_premium_1             22
pack_premium_2             22
create_date                22
modify_dat

**content_id nulls:** al ser esta variable el targer a recomendar, analizamos primero sus nulls y vemos que corresponden con contenido de "tutoriales" sobre el uso de la plataforma. Entonces se decide eliminar estas filas.

In [8]:
df[df['content_id'].isnull()]['category'].unique()

array(['Tutoriales/Decodificador Flow', nan], dtype=object)

In [9]:
df = df.dropna(subset=['content_id'])

**cast_first_name null:** esta es la variable con mayor cantidad de nulls, 28% de valores faltantes. Veremos el contenido de estas filas y tomaremos una decisión.
Vemos que en su mayoria se tratan de documentales o dibujos infantiles donde claramente no hay un autor para mencionar.
Se decide elimar esta columna.

In [10]:
df[df['cast_first_name'].isnull()]['category'].unique()

array(['Drama/Comedia', 'Reality', 'Infantil/Dibujos Animados',
       'Infantil/Animación', 'Documental', 'Documental/Viajes', 'Comedia',
       'Reality/Crimen', 'Documental/Religión', 'Documental/Naturaleza',
       'Documental/Historia', 'Drama', 'Aventura/Animación',
       'Entretenimiento/Música', 'Entretenimiento/Espectáculo',
       'Documental/Cultura', 'Documental/Crimen', 'Documental/Reality',
       'Infantil', 'Infantil/Aventura', 'Acción/Terror',
       'Reality/Documental', 'Documental/Biografía', 'Drama/Suspenso',
       'Entretenimiento/Restauración', 'Reality/Restauración',
       'Ciencia Ficción/Drama', 'Animación/Infantil', 'Acción/Suspenso',
       'Documental/Animales', 'Música/Entretenimiento',
       'Comedia/Animación', 'Reality/Naturaleza', 'Documental/Bélico',
       'Documental/Música', 'Reality/Cocina', 'Reality/Competencia',
       'Documental/Animación', 'Infantil/Comedia', 'Comedia/Infantil',
       'Comedia/Romance', 'Reality/Animales', 'Entretenimien

In [11]:
df = df.drop(columns = 'cast_first_name')

**credits_first_name nulls:** seguramente relacionada a la columna anterior. Tiene un 23% de nulos.
La revisamos y luego tomamos la decisión de eliminar la columna.

In [12]:
df[df['credits_first_name'].isnull()]['category'].unique()

array(['Drama', 'Terror', 'Drama/Romance', 'Reality',
       'Infantil/Dibujos Animados', 'Documental/Viajes',
       'Infantil/Comedia', 'Drama/Crimen', 'Entretenimiento/Restauración',
       'Reality/Moda', 'Entretenimiento/Cocina', 'Documental/Biografía',
       'Comedia', 'Reality/Cocina', 'Documental', 'Documental/Naturaleza',
       'Reality/Música', 'Entretenimiento/Música',
       'Entretenimiento/Espectáculo', 'Drama/Comedia',
       'Documental/Cultura', 'Acción/Crimen', 'Acción',
       'Documental/Crimen', 'Aventura/Fantasía',
       'Interés General/Deporte', 'Comedia/Familiar', 'Drama/Acción',
       'Reality/Crimen', 'Infantil/Animación', 'Infantil',
       'Entretenimiento/Competencia', 'Acción/Aventura', 'Drama/Fantasía',
       'Reality/Documental', 'Aventura/Familiar', 'Drama/Suspenso',
       'Entretenimiento/Moda', 'Magazine/Espectáculo',
       'Documental/Religión', 'Música/Documental', 'Western/Crimen',
       'Drama/Historia', 'Ciencia Ficción/Drama', 'Entreten

In [13]:
df = df.drop(columns = 'credits_first_name')

**episode_title nulls:** es la tercer variable con mayor cantidad de nulos ( 24% ). La revisamos y se decise eliminar la columna ya que este dato corresponde a series y nuestro recomendador no debe incluir capitulos de la misma serie ya vista.

In [14]:
df[df['episode_title'].isnull()]['show_type'].unique()

array(['Película', 'TV', 'Gaming', nan], dtype=object)

In [15]:
df = df.drop(columns = 'episode_title')

El resto de los nulos son muy pocos, entonces los eliminamos

In [16]:
df = df.dropna()

In [17]:
df.isnull().sum()

customer_id          0
account_id           0
device_type          0
asset_id             0
tunein               0
tuneout              0
resume               0
content_id           0
title                0
reduced_title        0
show_type            0
released_year        0
country_of_origin    0
category             0
keywords             0
description          0
reduced_desc         0
run_time_min         0
audience             0
made_for_tv          0
close_caption        0
sex_rating           0
violence_rating      0
language_rating      0
dialog_rating        0
fv_rating            0
pay_per_view         0
pack_premium_1       0
pack_premium_2       0
create_date          0
modify_date          0
start_vod_date       0
end_vod_date         0
dtype: int64

**Duplicados**

In [18]:
df = df.drop_duplicates(subset=None,keep='first',inplace=False,ignore_index=False)

**EDA**

**Histogramas**

In [19]:
columns = df.columns

#for c in columns:
    #if df[c].dtype == 'int64' or df[c].dtype == 'float64':
        #counts, bins = np.histogram(df[c])
        #plt.hist(bins[:-1],bins, weights=counts)
        #plt.title(f'Distribución: {c}')
        #plt.show()
    #else:
        #continue
    

**Bar chart**

In [20]:
#for c in columns:
    #try:
        #fig, axs = plt.subplots(1,figsize=(9, 3))
        #axs.bar(df[c].unique(), height = df[c].value_counts())
        #fig.suprtitle(f'Distribución:{c}')
        #plt.rcParams.update({'figure.max_open_warning': 0})
    #except:
        #print(f'{c} no se pudo procesar')

**Boxplot**

In [21]:
#for c in columns:
    #try:
        #plt.figure(figsize=(14,4))
        #sns.boxplot(x=df[c], y=df[c],palette= "Set2").set_title(f'Asset_id vs {c}')
    #except:
        #print(f'{c} no se pudo procesar')

**Ranking**

In [22]:
print("Top 20: Títulos")
print("---------------")
df['title'].value_counts().head(20)

Top 20: Títulos
---------------


T:7 Ep:44 Peppa Pig          13466
Cosa de minas                12279
T:1 Ep:01 El nudo             9941
T:5 Ep:05 This is Us          9692
T:5 Ep:06 This is Us          9385
T:1 Ep:01 The Collapse        8966
T:5 Ep:09 This is Us          8692
T:5 Ep:07 This is Us          8667
T:5 Ep:08 This is Us          7966
T:1 Ep:01 Big Little Lies     7149
T:1 Ep:02 El nudo             7102
3 metros sobre el cielo       6676
T:5 Ep:10 This is Us          6545
T:1 Ep:03 El nudo             6502
T:7 Ep:28 Peppa Pig           6170
T:1 Ep:04 El nudo             6160
Fuga de pretoria              6140
T:1 Ep:02 The Collapse        6132
T:1 Ep:01 Dime quién soy      5789
T:1 Ep:06 El nudo             5550
Name: title, dtype: int64

In [23]:
print("Top 20: Show_type")
print("-----------------")
df['show_type'].value_counts().head(20)

Top 20: Show_type
-----------------


Serie           1549966
TV              1178065
Película         867070
Web               32706
Rolling           26273
Gaming             2878
Series,Serie         54
Name: show_type, dtype: int64

In [24]:
print("Top 20: Categorías")
print("------------------")
df['category'].value_counts().head(20)

Top 20: Categorías
------------------


Infantil/Dibujos Animados    791790
Drama                        353797
Drama/Romance                296431
Drama/Comedia                211029
Drama/Crimen                 208368
Drama/Suspenso               146120
Comedia/Romance              136163
Infantil/Animación           113174
Comedia                      107923
Infantil/Comedia              99868
Acción/Drama                  77006
Acción/Crimen                 72761
Drama/Acción                  72248
Comedia/Drama                 64541
Aventura/Fantasía             50678
Comedia/Infantil              40473
Comedia/Familiar              39697
Infantil                      34829
Crimen/Drama                  31484
Reality/Cocina                30347
Name: category, dtype: int64

In [36]:
lista = df['show_type'].unique()
mask_lista = []

for l in lista:
    mask = df['show_type'] == l
    mask_lista.append(df[mask])

for ml in mask_lista:
    pd.DataFrame(df[ml]).to_csv(f'/datathonTelecom/Data/{l}.csv')

ValueError: Boolean array expected for the condition, not object